# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`CHEM-Data`*

---
*@author: Evan*\
*@date: 2023-06-06*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
isam = xr.open_dataset('F:/Data/Project_anqing/May/COMBINE_ACONC_v54_ISAM_intel_CN3AH_135X138_2023_isam.nc')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [2]:
files = ['F:/Data/Project_anqing/May/COMBINE_isam_15.nc',
         'F:/Data/Project_anqing/May/COMBINE_isam_16-31.nc']
isam = xr.open_mfdataset(files,combine='nested',concat_dim='TSTEP')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [4]:
# 将时间点转换为实际时间
times=pd.date_range('2023-05-16-00','2023-05-31-23',freq='h')
times

DatetimeIndex(['2023-05-16 00:00:00', '2023-05-16 01:00:00',
               '2023-05-16 02:00:00', '2023-05-16 03:00:00',
               '2023-05-16 04:00:00', '2023-05-16 05:00:00',
               '2023-05-16 06:00:00', '2023-05-16 07:00:00',
               '2023-05-16 08:00:00', '2023-05-16 09:00:00',
               ...
               '2023-05-31 14:00:00', '2023-05-31 15:00:00',
               '2023-05-31 16:00:00', '2023-05-31 17:00:00',
               '2023-05-31 18:00:00', '2023-05-31 19:00:00',
               '2023-05-31 20:00:00', '2023-05-31 21:00:00',
               '2023-05-31 22:00:00', '2023-05-31 23:00:00'],
              dtype='datetime64[ns]', length=384, freq='H')

In [5]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [6]:
days=1 # set spin-up days
dataset1=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        O3_AnQ=(['time','level','y','x'],isam.O3_AnQ[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing','units':'ug m-3'}),
        O3_AQI=(['time','level','y','x'],isam.O3_AQI[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Industry','units':'ug m-3'}),
        O3_AQT=(['time','level','y','x'],isam.O3_AQT[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Trasportation','units':'ug m-3'}),
        O3_AQA=(['time','level','y','x'],isam.O3_AQA[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_OTH=(['time','level','y','x'],isam.O3_OTH[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from OTHER','units':'ug m-3'}),
        O3_ICO=(['time','level','y','x'],isam.O3_ICO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from ICON','units':'ug m-3'}),
        O3_BCO=(['time','level','y','x'],isam.O3_BCO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset1

<xarray.Dataset>
Dimensions:    (time: 384, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-16 ... 2023-05-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    O3_AnQ     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    O3_AQI     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    O3_AQT     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    O3_AQA     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    O3_OTH     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    O3_ICO     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    O3_BCO     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [7]:
days=1 # set spin-up days
dataset2=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        PM_AnQ=(['time','level','y','x'],isam.PMI_AnQ[days*24-8:-8,:,:,:].data+isam.PMJ_AnQ[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing','units':'ug m-3'}),
        PM_AQI=(['time','level','y','x'],isam.PMI_AQI[days*24-8:-8,:,:,:].data+isam.PMJ_AQI[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing Industry','units':'ug m-3'}),
        PM_AQT=(['time','level','y','x'],isam.PMI_AQT[days*24-8:-8,:,:,:].data+isam.PMJ_AQT[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing Trasportation','units':'ug m-3'}),
        PM_AQA=(['time','level','y','x'],isam.PMI_AQA[days*24-8:-8,:,:,:].data+isam.PMJ_AQA[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing Agricultrue','units':'ug m-3'}),
        PM_OTH=(['time','level','y','x'],isam.PMI_OTH[days*24-8:-8,:,:,:].data+isam.PMJ_OTH[days*24-8:-8,:,:,:].data,{'long name':'PM25 from OTHER','units':'ug m-3'}),
        PM_ICO=(['time','level','y','x'],isam.PMI_ICO[days*24-8:-8,:,:,:].data+isam.PMJ_ICO[days*24-8:-8,:,:,:].data,{'long name':'PM25 from ICON','units':'ug m-3'}),
        PM_BCO=(['time','level','y','x'],isam.PMI_BCO[days*24-8:-8,:,:,:].data+isam.PMJ_BCO[days*24-8:-8,:,:,:].data,{'long name':'PM25 from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset2

<xarray.Dataset>
Dimensions:    (time: 384, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-16 ... 2023-05-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    PM_AnQ     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    PM_AQI     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    PM_AQT     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    PM_AQA     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    PM_OTH     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    PM_ICO     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
    PM_BCO     (time, level, y, x) float32 dask.array<chunksize=(8, 26, 138, 135), meta=np.ndarray>
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [8]:
compression=dict(zlib=True,complevel=5)

encoding={var:compression for var in dataset1.data_vars}
dataset1.to_netcdf('D:/Download/May_ISAM_O3-2.nc',encoding=encoding)

encoding={var:compression for var in dataset2.data_vars}
dataset2.to_netcdf('D:/Download/May_ISAM_PM-2.nc',encoding=encoding)